In [ ]:
%load_ext autoreload
%autoreload 2

from dataset import CustomImageDataset
import torch
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from neuralNetwork import NeuralNetwork

val_data = CustomImageDataset(dataset_path="../Datasets/face-emotion", mode="val", transform="scale_224")

val_dataloader = DataLoader(val_data, batch_size=8, shuffle=True)


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# model = NeuralNetwork().to(device)

import timm

model = timm.create_model("resnet18", pretrained=True, num_classes=2).to(device)


model.load_state_dict(torch.load("./models/resnet18.pt"))
model.eval()

In [ ]:
true_labels = []
predictions = []

# Iterate over the dataloader
for inputs, labels in val_dataloader:
    with torch.no_grad():  # No need to track gradients
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

    # Append true labels and predictions
    true_labels.extend(labels.argmax(-1).cpu().numpy())
    predictions.extend(predicted.cpu().numpy())

cm = confusion_matrix(true_labels, predictions)

sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
X, y = val_data.__getitem__(16)

X_cpu = X.cpu()
X_cpu = torch.reshape(X_cpu, (224,224,3))

X = X.unsqueeze(0)

logits = model(X)
pred_probab = torch.nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

if (y.argmax(-1) == 0):
    print(f"Actual class: happy")
if (y.argmax(-1) == 1):
    print(f"Actual class: sad") 
if (pred_probab.argmax(1)[0] == 0):
    print(f"Predicted class: happy")
if (pred_probab.argmax(1)[0] == 1):
    print(f"Predicted class: sad")

plt.imshow(X_cpu)
plt.show()
